<a href="https://colab.research.google.com/github/dhar174/video_gen_experiments/blob/main/Open_Sora_Plan_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone https://github.com/PKU-YuanGroup/Open-Sora-Plan
%cd Open-Sora-Plan

!pip install -e .
!pip install -e '.[dev]'
# !pip install --upgrade pillow



/content
Cloning into 'Open-Sora-Plan'...
remote: Enumerating objects: 12960, done.
remote: Counting objects: 100% (3413/3413), done.
remote: Compressing objects: 100% (803/803), done.
remote: Total 12960 (delta 3002), reused 2624 (delta 2610), pack-reused 9547 (from 1)
Receiving objects: 100% (12960/12960), 9.98 MiB | 13.70 MiB/s, done.
Resolving deltas: 100% (8553/8553), done.
/content/Open-Sora-Plan
Obtaining file:///content/Open-Sora-Plan
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.

Obtaining file:///content/Open-Sora-Plan
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 13.6 MB/s eta 0:00:00
  Building editable for opensora (pyproject.toml) ... done
  Created wheel for opensora: filename=opensora-1.3.0-0.editable-py3-none-any.whl size=14058 sha256=55a1120174824f0f9e8d45d7e9341f885ccc002ee7a6af3fd978f024bdcc7414
  Stored in directory: /tmp/pip-ephem-wheel-cache-goxbg4ja/wheels/34/5a/06/9c4c82fb5087d404edb42cbbeffaab828e35bd75b599d785b6
Successfully built opensora
  Attempting uninstall: opensora
    Found existing installation: opensora 1.3.0
    Uninstalling opensora-1.3.0:
      Successfully uninstalled opensora-1.3.0


In [2]:
# Add this line to ensure opensora is in the Python path
import sys
sys.path.append('/content/Open-Sora-Plan') # Assuming Open-Sora-Plan is cloned in /content


import os
# Get the current working directory
cwd = os.getcwd()
print(f"Current Working Directory: {cwd}")

# List the contents of the current directory
contents = os.listdir(cwd)
print("Contents of the Current Directory:")
for item in contents:
    print(item)
    if item == 'opensora':
      try:
        print(os.listdir(os.path.join(cwd, item)))
      except FileNotFoundError:
        print("Directory not found.")
      try:

        print(os.listdir(os.path.join(cwd, item, 'models')))
      except FileNotFoundError:
        print("Directory not found.")
      try:
        print(os.listdir(os.path.join(cwd, item, 'models', 'ae')))
      except FileNotFoundError:
        print("Directory not found.")
        print(os.listdir(os.path.join(cwd, item, 'models', 'diffusion')))
        try:
          print(os.listdir(os.path.join(cwd, item, 'models', 'causalvideovae')))
        except FileNotFoundError:
          print("Directory not found.")
        try:
          print(os.listdir(os.path.join(cwd, item, 'models', 'diffusion', 'latte')))
        except FileNotFoundError:
          print("Directory not found.")
      try:
        print(os.listdir(os.path.join(cwd, item, 'models', 'diffusion', 'latte')))
      except FileNotFoundError:
        print("Directory not found.")

Current Working Directory: /content/Open-Sora-Plan
Contents of the Current Directory:
pyproject.toml
opensora.egg-info
.gitignore
LICENSE
.github
examples
README.md
.git
docker
docs
opensora
['train', 'models', '__init__.py', 'acceleration', 'dataset', 'utils', 'serve', 'npu_config.py', 'sample', 'adaptor']
['causalvideovae', 'diffusion', '__init__.py', 'prompt_refiner', 'frame_interpolation', 'text_encoder', 'super_resolution']
Directory not found.
['opensora_v1_3', 'common.py', '__init__.py', 'curope']
['__init__.py', 'eval', 'dataset', 'utils', 'model', 'sample']
Directory not found.
Directory not found.
scripts


In [3]:



import random
import imageio
import torch
from diffusers import PNDMScheduler
from datetime import datetime
import gradio as gr
import numpy as np
from gradio.components import Textbox, Video, Image
from transformers import T5Tokenizer, T5EncoderModel

from opensora.models.causalvideovae import ae_stride_config, ae_wrapper
from opensora.models.diffusion.opensora_v1_3.modeling_opensora import OpenSoraT2V_v1_3
from opensora.sample.pipeline_opensora import OpenSoraPipeline
from opensora.serve.gradio_utils import block_css, title_markdown, randomize_seed_fn, set_env, examples, DESCRIPTION

def generate_img(prompt, sample_steps, scale, seed=0, randomize_seed=False, force_images=False):
    seed = int(randomize_seed_fn(seed, randomize_seed))
    set_env(seed)
    video_length = transformer_model.config.video_length if not force_images else 1
    height, width = int(args.version.split('x')[1]), int(args.version.split('x')[2])
    num_frames = 1 if video_length == 1 else int(args.version.split('x')[0])
    with torch.no_grad():
        videos = videogen_pipeline(prompt,
                                   video_length=video_length,
                                   height=height,
                                   width=width,
                                   num_inference_steps=sample_steps,
                                   guidance_scale=scale,
                                   enable_temporal_attentions=not force_images,
                                   num_images_per_prompt=1,
                                   mask_feature=True,
                                   ).video

    torch.cuda.empty_cache()
    videos = videos[0]
    tmp_save_path = 'tmp.mp4'
    imageio.mimwrite(tmp_save_path, videos, fps=24, quality=9)  # highest quality is 10, lowest is 0
    display_model_info = f"Video size: {num_frames}×{height}×{width}, \nSampling Step: {sample_steps}, \nGuidance Scale: {scale}"
    return tmp_save_path, prompt, display_model_info, seed

if __name__ == '__main__':
    args = type('args', (), {
        'ae': 'CausalVAEModel_4x8x8',
        'force_images': False,
        'model_path': 'LanguageBind/Open-Sora-Plan-v1.3.0',
        'text_encoder_name': 'DeepFloyd/t5-v1_1-xxl',
        'version': '65x512x512'
    })
    device = torch.device('cuda:0')

    # Load model:
    transformer_model = OpenSoraT2V_v1_3.from_pretrained(args.model_path, subfolder=args.version, torch_dtype=torch.float16, cache_dir='cache_dir').to(device)

    vae = ae_wrapper[args.ae](args.ae_path, **kwarg).eval().to(device)
    vae.vae.enable_tiling()
    image_size = int(args.version.split('x')[1])
    latent_size = (image_size // ae_stride_config[args.ae][1], image_size // ae_stride_config[args.ae][2])
    vae.latent_size = latent_size
    transformer_model.force_images = args.force_images
    tokenizer = T5Tokenizer.from_pretrained(args.text_encoder_name, cache_dir="cache_dir")
    text_encoder = T5EncoderModel.from_pretrained(args.text_encoder_name, cache_dir="cache_dir",
                                                  torch_dtype=torch.float16).to(device)

    # set eval mode
    transformer_model.eval()
    vae.eval()
    text_encoder.eval()
    scheduler = PNDMScheduler()
    videogen_pipeline = OpenSoraPipeline(vae=vae,
                                         text_encoder=text_encoder,
                                         tokenizer=tokenizer,
                                         scheduler=scheduler,
                                         transformer=transformer_model).to(device=device)


    demo = gr.Interface(
        fn=generate_img,
        inputs=[Textbox(label="",
                        placeholder="Please enter your prompt. \n"),
                gr.Slider(
                    label='Sample Steps',
                    minimum=1,
                    maximum=500,
                    value=50,
                    step=10
                ),
                gr.Slider(
                    label='Guidance Scale',
                    minimum=0.1,
                    maximum=30.0,
                    value=10.0,
                    step=0.1
                ),
                gr.Slider(
                    label="Seed",
                    minimum=0,
                    maximum=203279,
                    step=1,
                    value=0,
                ),
                gr.Checkbox(label="Randomize seed", value=True),
                gr.Checkbox(label="Generate image (1 frame video)", value=False),
                ],
        outputs=[Video(label="Vid", width=512, height=512),
                 Textbox(label="input prompt"),
                 Textbox(label="model info"),
                 gr.Slider(label='seed')],
        title=title_markdown, description=DESCRIPTION, theme=gr.themes.Default(), css=block_css,
        examples=examples,
    )
    demo.launch(share=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

ImportError: cannot import name 'is_directory' from 'PIL._util' (/usr/local/lib/python3.10/dist-packages/PIL/_util.py)

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install -U huggingface_hub --pre --upgrade
!pip install -U sentence-transformers==2.3.0
!pip install -q diffusers==0.24.0 gradio==3.50.2 einops==0.7.0 omegaconf==2.1.1 pytorch-lightning==1.4.2 torchmetrics==0.6.0 torchtext==0.6 accelerate==0.28.0 transformers

import os
import random
import imageio
import torch
from diffusers import PNDMScheduler, DiffusionPipeline
from datetime import datetime
import gradio as gr
import numpy as np
from gradio.components import Textbox, Video, Image
from transformers import T5Tokenizer, T5EncoderModel

# Removed imports from opensora since we're using DiffusionPipeline

def generate_img(prompt, sample_steps, scale, seed=0, randomize_seed=False, force_images=False):
    seed = int(randomize_seed_fn(seed, randomize_seed))
    set_env(seed)

    # Assuming pipe is defined globally
    image = pipe(prompt, num_inference_steps=sample_steps, guidance_scale=scale).images[0]

    tmp_save_path = 'tmp.png'  # Changed to png for single image
    image.save(tmp_save_path)
    display_model_info = f"Sampling Step: {sample_steps}, \nGuidance Scale: {scale}"
    return tmp_save_path, prompt, display_model_info, seed

if __name__ == '__main__':
    # Removed args since we're using from_pretrained
    device = torch.device('cuda:0')

    # Load model using DiffusionPipeline
    pipe = DiffusionPipeline.from_pretrained("LanguageBind/Open-Sora-Plan-v1.3.0", torch_dtype=torch.float16).to(device)

    # Removed manual loading of vae, text_encoder, tokenizer, scheduler
    # These are now handled by DiffusionPipeline

    demo = gr.Interface(
        # ... (rest of the Gradio interface code remains the same)
    )
    demo.launch(share=True)